In [1]:
import os
import pandas as pd

In [2]:
# Gather all filenames in provided directory
directory = "weather_data"
filenames = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
print(filenames)

['pressure.csv', 'wind_speed.csv', 'city_attributes.csv', 'humidity.csv', 'wind_direction.csv', 'weather_description.csv', 'temperature.csv']


In [3]:
# Create dataframe containing the Hour, Day, Month, and Year of all entries
df = pd.read_csv(os.path.join(directory, 'humidity.csv'))
df = pd.to_datetime(df['datetime'])
datetime_df = pd.DataFrame()
datetime_df['Hour'] = df.dt.hour
datetime_df['Day'] = df.dt.day
datetime_df['Month'] = df.dt.month
datetime_df['Year'] = df.dt.year
print(datetime_df)

       Hour  Day  Month  Year
0        12    1     10  2012
1        13    1     10  2012
2        14    1     10  2012
3        15    1     10  2012
4        16    1     10  2012
...     ...  ...    ...   ...
45248    20   29     11  2017
45249    21   29     11  2017
45250    22   29     11  2017
45251    23   29     11  2017
45252     0   30     11  2017

[45253 rows x 4 columns]


In [4]:
# Gather a dataframe for each csv file
all_feature_dfs = []
lat_long_df = pd.DataFrame()
for file in filenames:
    df = pd.read_csv(os.path.join(directory, file))
    if file != 'city_attributes.csv':
        df = df.drop(columns=['datetime', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya', 'Jerusalem'])
        all_feature_dfs.append((file[:len(file) - 4], df))
    else:
        df = df.drop(columns=['City', 'Country'])
        lat_long_df = df
    print(file)
    print(df)

pressure.csv
       Vancouver  Portland  San Francisco  Seattle  Los Angeles  San Diego  \
0            NaN       NaN            NaN      NaN          NaN        NaN   
1            NaN    1024.0         1009.0   1027.0       1013.0     1013.0   
2            NaN    1024.0         1009.0   1027.0       1013.0     1013.0   
3            NaN    1024.0         1009.0   1028.0       1013.0     1013.0   
4            NaN    1024.0         1009.0   1028.0       1013.0     1013.0   
...          ...       ...            ...      ...          ...        ...   
45248        NaN    1031.0            NaN   1030.0       1016.0     1017.0   
45249        NaN    1030.0            NaN   1030.0       1016.0     1017.0   
45250        NaN    1030.0            NaN   1029.0       1015.0     1016.0   
45251        NaN    1029.0            NaN   1028.0       1016.0     1016.0   
45252        NaN    1029.0            NaN   1028.0       1015.0     1017.0   

       Las Vegas  Phoenix  Albuquerque  Denver  ..

In [5]:
# Combine all features into a dataframe for each city
all_city_dfs = []
for city_idx in range(30):
    city_df = datetime_df.copy()
    for feature, df in all_feature_dfs:
        city_df[feature] = df.iloc[:, city_idx]
    city_name = df.columns[city_idx]
    city_df['Latitude'] = lat_long_df.iloc[city_idx, 0]
    city_df['Longitude'] = lat_long_df.iloc[city_idx, 1]
    print(city_name)
    print(city_df)
    all_city_dfs.append(city_df)

Vancouver
       Hour  Day  Month  Year  pressure  wind_speed  humidity  wind_direction  \
0        12    1     10  2012       NaN         NaN       NaN             NaN   
1        13    1     10  2012       NaN         0.0      76.0             0.0   
2        14    1     10  2012       NaN         0.0      76.0             6.0   
3        15    1     10  2012       NaN         0.0      76.0            20.0   
4        16    1     10  2012       NaN         0.0      77.0            34.0   
...     ...  ...    ...   ...       ...         ...       ...             ...   
45248    20   29     11  2017       NaN         NaN       NaN             NaN   
45249    21   29     11  2017       NaN         NaN       NaN             NaN   
45250    22   29     11  2017       NaN         NaN       NaN             NaN   
45251    23   29     11  2017       NaN         NaN       NaN             NaN   
45252     0   30     11  2017       NaN         NaN       NaN             NaN   

      weather_des

In [6]:
# Combine all city dataframes into a single master dataframe
master_df = pd.concat(all_city_dfs, axis=0)
print(master_df)

       Hour  Day  Month  Year  pressure  wind_speed  humidity  wind_direction  \
0        12    1     10  2012       NaN         NaN       NaN             NaN   
1        13    1     10  2012       NaN         0.0      76.0             0.0   
2        14    1     10  2012       NaN         0.0      76.0             6.0   
3        15    1     10  2012       NaN         0.0      76.0            20.0   
4        16    1     10  2012       NaN         0.0      77.0            34.0   
...     ...  ...    ...   ...       ...         ...       ...             ...   
45248    20   29     11  2017    1017.0         8.0      37.0           290.0   
45249    21   29     11  2017    1019.0         6.0      74.0           340.0   
45250    22   29     11  2017    1019.0         7.0      74.0           340.0   
45251    23   29     11  2017    1022.0         2.0      56.0           330.0   
45252     0   30     11  2017    1023.0         2.0      56.0           320.0   

      weather_description  

In [7]:
# Save to a csv file
master_df.to_csv('all_weather_data.csv')